In [5]:
# %%bash
# mkdir tmp
# FILE=swivel.tar.gz
# wget --quiet -O tmp/swivel.tar.gz  https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1?tf-hub-format=compressed
# cd tmp
# tar xvfz swivel.tar.gz
# cd ..
# mv tmp swivel
# gsutil -m cp -R swivel gs://trendspotting-pipeline/swivel

assets/
assets/tokens.txt
saved_model.pb
variables/
variables/variables.data-00000-of-00001
variables/variables.index


Copying file://swivel/swivel.tar.gz [Content-Type=application/x-tar]...
Copying file://swivel/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://swivel/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://swivel/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://swivel/assets/tokens.txt [Content-Type=text/plain]...
/ [5/5 files][  3.2 MiB/  3.2 MiB] 100% Done                                    
Operation completed over 5 objects/3.2 MiB.                                      


In [8]:
%%bigquery
CREATE OR REPLACE MODEL trendspotting.swivel_text_embed
OPTIONS(model_type='tensorflow', model_path='gs://trendspotting-pipeline/swivel/*')

Query complete after 0.01s: 100%|██████████| 2/2 [00:00<00:00, 823.62query/s]                        


""


In [35]:
%%bigquery
CREATE TEMPORARY FUNCTION arr_to_input_20(arr ARRAY<FLOAT64>)
RETURNS 
STRUCT<p1 FLOAT64, p2 FLOAT64, p3 FLOAT64, p4 FLOAT64,
       p5 FLOAT64, p6 FLOAT64, p7 FLOAT64, p8 FLOAT64, 
       p9 FLOAT64, p10 FLOAT64, p11 FLOAT64, p12 FLOAT64, 
       p13 FLOAT64, p14 FLOAT64, p15 FLOAT64, p16 FLOAT64,
       p17 FLOAT64, p18 FLOAT64, p19 FLOAT64, p20 FLOAT64>
AS (
STRUCT(
    arr[OFFSET(0)]
    , arr[OFFSET(1)]
    , arr[OFFSET(2)]
    , arr[OFFSET(3)]
    , arr[OFFSET(4)]
    , arr[OFFSET(5)]
    , arr[OFFSET(6)]
    , arr[OFFSET(7)]
    , arr[OFFSET(8)]
    , arr[OFFSET(9)]
    , arr[OFFSET(10)]
    , arr[OFFSET(11)]
    , arr[OFFSET(12)]
    , arr[OFFSET(13)]
    , arr[OFFSET(14)]
    , arr[OFFSET(15)]
    , arr[OFFSET(16)]
    , arr[OFFSET(17)]
    , arr[OFFSET(18)]
    , arr[OFFSET(19)]    
));

CREATE OR REPLACE MODEL trendspotting.embed_clustering_100
OPTIONS(model_type='kmeans', NUM_CLUSTERS=100, kmeans_init_method = 'KMEANS++') AS
SELECT
  arr_to_input_20(output_0) AS comments_embed
FROM ML.PREDICT(MODEL trendspotting.swivel_text_embed,(
  SELECT date, geo_name, term AS sentences, volume
  FROM `cpg-cdp.trendspotting.futurama_weekly`
  WHERE date between '2019-01-01' and '2021-12-31' and category_id = 10047
))

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1289.36query/s]                        


""


### Predict swivel embedding clusters

In [36]:
%%bigquery
CREATE TEMPORARY FUNCTION arr_to_input_20(arr ARRAY<FLOAT64>)
RETURNS 
STRUCT<p1 FLOAT64, p2 FLOAT64, p3 FLOAT64, p4 FLOAT64,
       p5 FLOAT64, p6 FLOAT64, p7 FLOAT64, p8 FLOAT64, 
       p9 FLOAT64, p10 FLOAT64, p11 FLOAT64, p12 FLOAT64, 
       p13 FLOAT64, p14 FLOAT64, p15 FLOAT64, p16 FLOAT64,
       p17 FLOAT64, p18 FLOAT64, p19 FLOAT64, p20 FLOAT64>
AS (
STRUCT(
    arr[OFFSET(0)]
    , arr[OFFSET(1)]
    , arr[OFFSET(2)]
    , arr[OFFSET(3)]
    , arr[OFFSET(4)]
    , arr[OFFSET(5)]
    , arr[OFFSET(6)]
    , arr[OFFSET(7)]
    , arr[OFFSET(8)]
    , arr[OFFSET(9)]
    , arr[OFFSET(10)]
    , arr[OFFSET(11)]
    , arr[OFFSET(12)]
    , arr[OFFSET(13)]
    , arr[OFFSET(14)]
    , arr[OFFSET(15)]
    , arr[OFFSET(16)]
    , arr[OFFSET(17)]
    , arr[OFFSET(18)]
    , arr[OFFSET(19)]    
));

CREATE OR REPLACE TABLE trendspotting.futurama_weekly_embed_cluster_100 as (
    select * 
    from ML.PREDICT(MODEL trendspotting.embed_clustering_100, (
        select *, arr_to_input_20(output_0) AS comments_embed from 
            ML.PREDICT(MODEL trendspotting.swivel_text_embed,(
          SELECT date, geo_name, term AS sentences, volume
          FROM `cpg-cdp.trendspotting.futurama_weekly`
          WHERE date between '2019-01-01' and '2021-12-31' and category_id = 10047
        ))
        )
    )
)

Query complete after 0.00s: 100%|██████████| 45/45 [00:00<00:00, 20997.18query/s]                      


""


In [37]:
%%bigquery

CREATE OR REPLACE TABLE trendspotting.futurama_weekly_embed_cluster_agg_100 as (
with centroids as (select * from 
(SELECT
centroid_id, feature, numerical_value
FROM
  ML.CENTROIDS(MODEL `trendspotting.embed_clustering_100`)
)
PIVOT(sum(numerical_value) for feature in ('comments_embed_p1',
'comments_embed_p2',
'comments_embed_p3',
'comments_embed_p4',
'comments_embed_p5',
'comments_embed_p6',
'comments_embed_p7',
'comments_embed_p8',
'comments_embed_p9',
'comments_embed_p10',
'comments_embed_p11',
'comments_embed_p12',
'comments_embed_p13',
'comments_embed_p14',
'comments_embed_p15',
'comments_embed_p16',
'comments_embed_p17',
'comments_embed_p18',
'comments_embed_p19',
'comments_embed_p20'))
                  )
select volume, date, b.*,
case when date between '2019-01-01' and  '2020-12-31' then 'TRAIN'
          when date between '2021-01-01' and '2021-05-31' then 'VALIDATE'
         else 'TEST' end as split_col
from (
    select sum(volume) as volume, date, centroid_id 
    from trendspotting.futurama_weekly_embed_cluster_100 group by date, centroid_id
) a
inner join centroids b on a.centroid_id = b.centroid_id
)

Query complete after 0.00s: 100%|██████████| 7/7 [00:00<00:00, 4051.35query/s]                        


""


In [ ]:
create table trendspotting.cluster_forecast_test_clean as (
    SELECT centroid_id, cast(volume as int) as volume, cast(predicted_volume.value as float64) as predicted_volume, date, predicted_on_date  FROM `cpg-cdp.trendspotting.cluster_forecast_test` 
)